**Part 1: Viterbi Algorithm**

In [75]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

obsProbsData = pd.read_csv("http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv")
transProbsData = pd.read_csv("http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv")
testData = pd.read_csv("http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv")
transProbsData

,P(x|...),C,H,START
0,C,0.87,0.47,0.5
1,H,0.07,0.47,0.5
2,STOP,0.06,0.06,0.0


In [76]:
obsProbs = np.array(obsProbsData.iloc[:])
transProbs = np.array(transProbsData.iloc[:])
obsProbs

array([[1.000e+00, 6.407e-01, 2.000e-04],
       [2.000e+00, 1.481e-01, 5.341e-01],
       [3.000e+00, 2.122e-01, 4.657e-01]])

In [77]:
def predictSeq(x):
  rawSeq = np.array(testData.iloc[x])[1:]
  # remove all the zeros
  seq = rawSeq[rawSeq != 0]
  # print(seq)
  mValues = []
  cValues = []
  for i in range(len(seq)):
    iceCream = seq[i]
    # print(iceCream)
    # mPair = [M_(i, COLD), M_(i, HOT)]
    mPair = [0, 0]
    # cPair = [most likely for day x given day x+1 is cold,
    #          most likely for day x given day x+2 is hot]
    #          0 = cold, 1 = hot
    cPair = [0, 0]
    # initialization step of Viterbi algorithm
    if i == 0:
      # transProbs[0][-1] = prior probability of cold
      # transProbs[1][-1] = prior probability of hot
      # obsProbs[x-1][1] = probability of x ice cream cones given cold
      # obsProbs[x-1][2] = probability of x ice cream cones given hot
      mPair[0] = transProbs[0][-1]*obsProbs[iceCream-1][1]
      mPair[1] = transProbs[1][-1]*obsProbs[iceCream-1][2]
    # recursive step
    else:
      # print(i)
      # transProbs[0][1] = probability today is cold given yesterday was cold
      # transProbs[1][1] = probability today is hot given yesterday was cold
      # you probably get the idea
      mPair[0] = obsProbs[iceCream-1][1] * max(transProbs[0][1]*mValues[i-1][0],
          transProbs[0][2]*mValues[i-1][1])
      mPair[1] = obsProbs[iceCream-1][2] * max(transProbs[1][1]*mValues[i-1][0],
          transProbs[1][2]*mValues[i-1][1])
      if transProbs[0][2]*mValues[i-1][1] > transProbs[0][1]*mValues[i-1][0]:
        cPair[0] = 1
      if transProbs[1][2]*mValues[i-1][1] > transProbs[1][1]*mValues[i-1][0]:
        cPair[1] = 1
      cValues.append(cPair)
          
    mValues.append(mPair)
    # print(mPair)
    # print(mValues)
    # print(cValues)

  # now for the predictions section
  # first, predict the final day's weather using M values
  predictions = []
  if mValues[-1][1] > mValues[-1][0]:
    predictions.append(1)
  else:
    predictions.append(0)
  # then, predict all other days using C values
  for i in reversed(range(len(cValues))):
    predictions.insert(0, cValues[i][predictions[-1]])
  return predictions
  
# printing results now
def printPredictions(x):
  print("Sequence #{}".format(x))
  rawSeq = np.array(testData.iloc[x])[1:]
  seq = rawSeq[rawSeq != 0]
  predictions = predictSeq(x)
  temps = ["cold", "hot"]
  for i in range(len(seq)):
    print("Day {}, {} ice cream cones, predicted weather {}".format(
        i+1, seq[i], temps[predictions[i]]))
  
for i in range(len(testData)):
  printPredictions(i)
  print()

Sequence #0
Day 1, 2 ice cream cones, predicted weather hot
Day 2, 3 ice cream cones, predicted weather hot
Day 3, 3 ice cream cones, predicted weather hot
Day 4, 2 ice cream cones, predicted weather hot
Day 5, 3 ice cream cones, predicted weather hot

Sequence #1
Day 1, 2 ice cream cones, predicted weather hot
Day 2, 3 ice cream cones, predicted weather hot
Day 3, 2 ice cream cones, predicted weather hot
Day 4, 2 ice cream cones, predicted weather hot

Sequence #2
Day 1, 3 ice cream cones, predicted weather hot
Day 2, 1 ice cream cones, predicted weather cold
Day 3, 3 ice cream cones, predicted weather cold
Day 4, 3 ice cream cones, predicted weather cold
Day 5, 1 ice cream cones, predicted weather cold

Sequence #3
Day 1, 2 ice cream cones, predicted weather hot
Day 2, 1 ice cream cones, predicted weather cold
Day 3, 1 ice cream cones, predicted weather cold

Sequence #4
Day 1, 1 ice cream cones, predicted weather cold
Day 2, 1 ice cream cones, predicted weather cold
Day 3, 1 ice cre

As we can see here, if at least one day had only one ice cream cone, that day and every day after that will be predicted as cold. This is because on days with one ice cream cone, the probability of cold weather is overwhelmingly greater than the probability of hot weather. Aside from that, any sequence without days with only one ice cream cone is predicted entirely as hot.

**Part 2: Approximate Inference**

In [78]:
# generates a sequence of days by weather based solely on prior/transition probabilities
def generateWeatherSeq(length):
  seq = []
  for i in range(length):
    weather = 0
    if i == 0:
      if np.random.random() > transProbs[0][-1]:
        weather = 1
    else:
      if np.random.random() > transProbs[0][seq[-1]+1]:
        weather = 1
    seq.append(weather)
  return seq

for i in range(100):
  print(generateWeatherSeq(5))

[0, 0, 1, 0, 0]
[1, 0, 0, 0, 1]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[1, 1, 1, 0, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 1]
[0, 0, 0, 0, 0]
[1, 0, 0, 1, 1]
[0, 1, 1, 0, 0]
[0, 0, 0, 0, 1]
[1, 1, 0, 0, 0]
[0, 0, 1, 1, 1]
[0, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 1, 1]
[0, 0, 0, 0, 0]
[0, 0, 1, 1, 1]
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
[1, 0, 0, 1, 1]
[1, 0, 0, 0, 1]
[0, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 1]
[0, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 0, 1, 1]
[1, 1, 1, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 1]
[1, 1, 1, 1, 1]
[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[0, 0, 0, 0, 1]
[1, 1, 0, 0, 1]
[0, 0, 0, 1, 0]
[1, 1, 1, 1, 0]
[1, 0, 0, 1, 0]
[1, 1, 0, 0, 0]
[0, 0, 0

In [81]:
# gets the weight of a sequence of temperatures, based on a test sample
def getWeight(seq, sample):
  weight = 1.0
  for i in range(len(seq)):
    weather = seq[i]
    iceCream = sample[i]
    # for reference again:
    # obsProbs[x-1][1] = probability of x ice cream cones given cold
    # obsProbs[x-1][2] = probability of x ice cream cones given hot
    weight *= obsProbs[iceCream-1][weather+1]
    # print("weight: {}".format(weight))
  return weight

rawSample = np.array(testData.iloc[1])[1:]
sample = rawSample[rawSample != 0]
seq = generateWeatherSeq(sample.size)
print(seq)
print(sample)
print(getWeight(seq, sample))

[0, 0, 0, 0]
[2 3 2 2]
0.0006893036134202001


In [82]:
import operator

# makes a dictionary of sequence frequency counts multipled by weights
# for easy probability calculation
def seqFreqsAndWeights(sample, iters):
  freqs = {}
  for i in range(iters):
    seq = generateWeatherSeq(sample.size)
    strSeq = str(seq)
    # print(strSeq)
    if strSeq in freqs:
      freqs[strSeq] += getWeight(seq, sample)
    else:
      freqs[strSeq] = getWeight(seq, sample)
  return freqs

# unused
def seqProb(seq, dict):
  return dict[seq]/sum(dict.values())

# now to list the most likely predictions of each sample, with both algorithms
for i in range(len(testData)):
  rawSample = np.array(testData.iloc[i])[1:]
  sample = rawSample[rawSample != 0]
  freqs = seqFreqsAndWeights(sample, 10000)
  temps = ["C", "H"]
  print("Sample: {}".format(sample))
  pred_viterbi = predictSeq(i)
  pred_approx = max(freqs, key = freqs.get)
  print("Most likely sequence (Viterbi algorithm): {}".format(pred_viterbi))
  print("Most likely sequence (approximate inference): {}".format(pred_approx))
  print()

Sample: [2 3 3 2 3]
Most likely sequence (Viterbi algorithm): [1, 1, 1, 1, 1]
Most likely sequence (approximate inference): [1, 1, 1, 1, 1]

Sample: [2 3 2 2]
Most likely sequence (Viterbi algorithm): [1, 1, 1, 1]
Most likely sequence (approximate inference): [1, 1, 1, 1]

Sample: [3 1 3 3 1]
Most likely sequence (Viterbi algorithm): [1, 0, 0, 0, 0]
Most likely sequence (approximate inference): [1, 0, 0, 0, 0]

Sample: [2 1 1]
Most likely sequence (Viterbi algorithm): [1, 0, 0]
Most likely sequence (approximate inference): [1, 0, 0]

Sample: [1 1 1 2 3]
Most likely sequence (Viterbi algorithm): [0, 0, 0, 0, 0]
Most likely sequence (approximate inference): [0, 0, 0, 0, 0]

Sample: [1 3 1 1]
Most likely sequence (Viterbi algorithm): [0, 0, 0, 0]
Most likely sequence (approximate inference): [0, 0, 0, 0]

Sample: [3 2 3]
Most likely sequence (Viterbi algorithm): [1, 1, 1]
Most likely sequence (approximate inference): [1, 1, 1]

Sample: [2 2 1 1]
Most likely sequence (Viterbi algorithm): [

(Note that in the above results, 0 = cold, 1 = hot.)

Most of the predicted weather sequences were the same between the Viterbi algorithm and approximate inference. The only sequence that wasn't the same was from the ice cream sample [1, 3, 2, 3, 3]. The Viterbi algorithm predicted [0, 0, 0, 0, 0] (all days cold), whereas approximate inference predicted [0, 1, 1, 1, 1] (day 1 cold, days 2-5 hot). This is probably because the cold to cold transition probability is very low, and thus samples with all days cold were infrequent enough that it made up for the extremely low weight of samples where at least one day was hot.

Note that the above results were from testing with 10,000 iterations. With fewer iterations, there was somewhat more variance between the Vitberbi and approximate inference results.